# Mozio Boarding




## Parameters

In [ ]:
#source
lpaInputContainer = 'silver'
lpaInputStorageAccount = 'mxdatalakedevcc001.dfs.core.windows.net'
lpaInputPath = 'digital/mozio/Mozio Presto/Mozio_Boarding/'
lpaInputFullFileName = 'Mozio_Presto_Boardings.csv'
lpaInputFormat = 'csv'

#source reference 
lpaInputReferenceFilePath = 'digital/mozio/reference'
lpaInputReferenceHolidaysFile = 'Holidays.csv'
lpaReferenceFileFormat = 'csv'

#target
lpaOutputContainer = 'silver'
lpaOutputStorageAccount = 'mxdatalakedevcc001.dfs.core.windows.net'
lpaOutputPath = 'digital/mozio/Mozio_Presto_Output/BoardingNotebook_Output'
lpaOutputFormat = 'parquet'
lpaOutputMode = 'append'

#spark configuration
lpaResetSparkTimeParserPolicy = True

## Code

In [ ]:
from os import path

sourceFile = 'abfss://' + path.normpath(path.join(f'{lpaInputContainer}@{lpaInputStorageAccount}', lpaInputPath, lpaInputFullFileName))

targetFolderPath = 'abfss://' + path.normpath(path.join(f'{lpaOutputContainer}@{lpaOutputStorageAccount}/{lpaOutputPath}'))

sourceReferenceHolidaysFile = 'abfss://' + path.normpath(path.join(f'{lpaInputContainer}@{lpaInputStorageAccount}', lpaInputReferenceFilePath, lpaInputReferenceHolidaysFile))


In [ ]:
#df = spark.read.load('abfss://silver@mxdatalakedevcc001.dfs.core.windows.net/digital/mozio/Mozio Presto/Mozio_Boarding/Mozio_Presto_Boardings.csv', format='csv'
df = spark.read.load(sourceFile, format=lpaInputFormat, header=True)


# In[2]:
df1 = df.filter(df.activation_date != "null")

In [ ]:
from pyspark.sql.types import IntegerType
df1 = df.withColumn("Hour_24", df["Hour_24"].cast(IntegerType()))\
        .withColumn("DayOfWeek", df["DayOfWeek"].cast(IntegerType()))

In [ ]:
from pyspark.sql.functions import when,col

df1 = df1.withColumn('DayOfWeek', when(((col('Hour_24')>=0) & (col('Hour_24')<=4)) & (col('DayOfWeek')>1), col('DayOfWeek')-1)
         .otherwise(col('DayOfWeek')))


#.otherwise(when(((col('Hour_24')>=0) & (col('Hour_24')<=4)) & (col('DayOfWeek')==1),col('DayOfWeek')+6)

In [ ]:
from pyspark.sql.functions import split, col
from pyspark.sql import functions as F
df3 = df1.withColumn("Boardings", df1['ticket_passengers'])\
         .withColumn("Revenue",df1['amount_paid_cad'])
       

In [ ]:
from pyspark.sql.functions import when,col
df3 = df3.withColumn('Boardings', when(col('ticket_code')=='weekday_group_pass_2_pax', col('ticket_passengers')*2).when(col('ticket_code')=='weekday_group_pass_3_pax', col('ticket_passengers')*3).when(col('ticket_code')=='weekday_group_pass_4_pax', col('ticket_passengers')*4).when(col('ticket_code')=='weekday_group_pass_5_pax', col('ticket_passengers')*5)
                     .otherwise(col('ticket_passengers')))

In [ ]:
from pyspark.sql.functions import col, unix_timestamp, to_date
df4 = df3.withColumn('activation_date', 
                   to_date(unix_timestamp(col('activation_date'), 'dd-MMM-yyyy').cast("timestamp")))

In [ ]:
#sourceReferenceHolidaysFile = 'abfss://silver@mxdatalakedevcc001.dfs.core.windows.net/digital/mozio/reference/Holidays.csv'
df2 = spark.read.load(sourceReferenceHolidaysFile, format=lpaReferenceFileFormat, header=True)

In [ ]:
from pyspark.sql.functions import col, unix_timestamp, to_date
df_Holiday = df2.withColumn('Holidays', 
                   to_date(unix_timestamp(col('Holidays'), "M/d/yyyy").cast("timestamp")))

In [ ]:
pandas_holiday = df_Holiday.toPandas()

In [ ]:
#df4.printSchema()

In [ ]:
# corrections are added to allow execution on dev environment
if lpaResetSparkTimeParserPolicy:
    spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

if 'FYear' not in df4.schema.fieldNames():
    df4 = df4.withColumnRenamed('FY','FYear')

if 'Service_Provider' not in df4.schema.fieldNames():
    df4 = df4.withColumnRenamed('Transfer_Service_Provider','Service_Provider')

In [ ]:
pandasDF = df4.toPandas()

In [ ]:
#display(pandasDF)

In [ ]:
#Part-B/Logic for Friday Holidays
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np 
temp_Fri = pd.DataFrame()
temp_Fri['ticket_code'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), "full_weekend_pass", np.nan)
temp_Fri['Universal_Class'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), "full_weekend_pass", np.nan)
temp_Fri['Year'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Year'], np.nan)
temp_Fri['Month'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Month'], np.nan)
temp_Fri['Day'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Day'], np.nan)
temp_Fri['Hour_24'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Hour_24'], np.nan)
temp_Fri['Minute'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Minute'], np.nan)
temp_Fri['FYear'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['FYear'], np.nan)
temp_Fri['Week'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Week'], np.nan)
temp_Fri['DayOfWeek'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['DayOfWeek'], np.nan)
temp_Fri['Holidays'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Holidays'], np.nan)
temp_Fri['Time_period'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Time_period'], np.nan)
temp_Fri['stop_lat'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['stop_lat'], np.nan)
temp_Fri['stop_lon'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['stop_lon'], np.nan)
temp_Fri['Mode_Type'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Mode_Type'], np.nan)
temp_Fri['activation_date'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['activation_date'], np.nan)
temp_Fri['activation_time'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), "Unknown", np.nan)
temp_Fri['ticket_passengers'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['ticket_passengers'], np.nan)
temp_Fri['Origin'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Destination'], np.nan)
temp_Fri['Destination'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Origin'], np.nan)
temp_Fri['amount_paid_cad'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['amount_paid_cad'], np.nan)
temp_Fri['Revenue'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), "0", np.nan)
temp_Fri['Boardings'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Boardings'], np.nan)

temp_Fri['Service_Provider'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Service_Provider'], np.nan)
temp_Fri['Ticket_System_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Ticket_System_Name'], np.nan)
temp_Fri['Line_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Line_Name'], np.nan)

                                   
 

#df_list.concat(temp)
temp_Fri=temp_Fri[pd.notnull(temp_Fri['ticket_code'])] 

In [ ]:
#Part-B/Logic for Friday Holidays 
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np 
temp_Fri_oneday = pd.DataFrame()
temp_Fri_oneday['ticket_code'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), "full_weekend_pass", np.nan)
temp_Fri_oneday['Universal_Class'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), "full_weekend_pass", np.nan)
temp_Fri_oneday['Year'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Year'], np.nan)
temp_Fri_oneday['Month'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Month'], np.nan)
temp_Fri_oneday['Day'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Day'], np.nan)
temp_Fri_oneday['Hour_24'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Hour_24'], np.nan)
temp_Fri_oneday['Minute'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Minute'], np.nan)
temp_Fri_oneday['FYear'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['FYear'], np.nan)
temp_Fri_oneday['Week'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Week'], np.nan)
temp_Fri_oneday['DayOfWeek'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['DayOfWeek'], np.nan)
temp_Fri_oneday['Holidays'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Holidays'], np.nan)
temp_Fri_oneday['Time_period'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Time_period'], np.nan)
temp_Fri_oneday['stop_lat'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['stop_lat'], np.nan)
temp_Fri_oneday['stop_lon'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['stop_lon'], np.nan)
temp_Fri_oneday['Mode_Type'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Mode_Type'], np.nan)
temp_Fri_oneday['activation_date'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['activation_date']+ pd.Timedelta(days=1), np.nan)

temp_Fri_oneday['activation_time'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), "Unknown", np.nan)
temp_Fri_oneday['ticket_passengers'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['ticket_passengers'], np.nan)
temp_Fri_oneday['Origin'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Origin'], np.nan)
temp_Fri_oneday['Destination'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Destination'], np.nan)
temp_Fri_oneday['amount_paid_cad'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['amount_paid_cad'], np.nan)
temp_Fri_oneday['Revenue'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), "0", np.nan)
temp_Fri_oneday['Boardings'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Boardings'], np.nan)

temp_Fri_oneday['Service_Provider'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Service_Provider'], np.nan)
temp_Fri_oneday['Ticket_System_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Ticket_System_Name'], np.nan)
temp_Fri_oneday['Line_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Line_Name'], np.nan)

                                   
 

#df_list.concat(temp)
temp_Fri_oneday=temp_Fri_oneday[pd.notnull(temp_Fri_oneday['ticket_code'])] 



# In[15]:


#Part-B/Logic for Friday Holidays 
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np 
temp_Fri_oneday_R = pd.DataFrame()
temp_Fri_oneday_R['ticket_code'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), "full_weekend_pass", np.nan)
temp_Fri_oneday_R['Universal_Class'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), "full_weekend_pass", np.nan)
temp_Fri_oneday_R['Year'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Year'], np.nan)
temp_Fri_oneday_R['Month'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Month'], np.nan)
temp_Fri_oneday_R['Day'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Day'], np.nan)
temp_Fri_oneday_R['Hour_24'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Hour_24'], np.nan)
temp_Fri_oneday_R['Minute'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Minute'], np.nan)
temp_Fri_oneday_R['FYear'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['FYear'], np.nan)
temp_Fri_oneday_R['Week'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Week'], np.nan)
temp_Fri_oneday_R['DayOfWeek'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['DayOfWeek'], np.nan)
temp_Fri_oneday_R['Holidays'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Holidays'], np.nan)
temp_Fri_oneday_R['Time_period'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Time_period'], np.nan)
temp_Fri_oneday_R['stop_lat'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['stop_lat'], np.nan)
temp_Fri_oneday_R['stop_lon'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['stop_lon'], np.nan)
temp_Fri_oneday_R['Mode_Type'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Mode_Type'], np.nan)
temp_Fri_oneday_R['activation_date'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['activation_date']+ pd.Timedelta(days=1), np.nan)

temp_Fri_oneday_R['activation_time'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), "Unknown", np.nan)
temp_Fri_oneday_R['ticket_passengers'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['ticket_passengers'], np.nan)
temp_Fri_oneday_R['Origin'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Destination'], np.nan)
temp_Fri_oneday_R['Destination'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Origin'], np.nan)
temp_Fri_oneday_R['amount_paid_cad'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['amount_paid_cad'], np.nan)
temp_Fri_oneday_R['Revenue'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), "0", np.nan)
temp_Fri_oneday_R['Boardings'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Boardings'], np.nan)

temp_Fri_oneday_R['Service_Provider'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Service_Provider'], np.nan)
temp_Fri_oneday_R['Ticket_System_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Ticket_System_Name'], np.nan)
temp_Fri_oneday_R['Line_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '6'), pandasDF['Line_Name'], np.nan)

                                   
 

#df_list.concat(temp)
temp_Fri_oneday_R=temp_Fri_oneday_R[pd.notnull(temp_Fri_oneday_R['ticket_code'])] 

#temp_Fri,temp_Fri_oneday,temp_Fri_oneday_R

In [ ]:
#Part-C for Saturday 
import pandas as pd
import numpy as np 
temp_sat = pd.DataFrame()

#temp_sat['ticket_code'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), "full_weekend_pass", np.nan)

temp_sat['ticket_code'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), "full_weekend_pass", np.nan)
temp_sat['Universal_Class'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), "full_weekend_pass", np.nan)
temp_sat['Year'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Year'], np.nan)
temp_sat['Month'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Month'], np.nan)
temp_sat['Day'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Day'], np.nan)
temp_sat['Hour_24'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Hour_24'], np.nan)
temp_sat['Minute'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Minute'], np.nan)
temp_sat['FYear'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['FYear'], np.nan)
temp_sat['Week'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Week'], np.nan)
temp_sat['DayOfWeek'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['DayOfWeek'], np.nan)
temp_sat['Holidays'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Holidays'], np.nan)
temp_sat['Time_period'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Time_period'], np.nan)
temp_sat['stop_lat'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['stop_lat'], np.nan)
temp_sat['stop_lon'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['stop_lon'], np.nan)
temp_sat['Mode_Type'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Mode_Type'], np.nan)
temp_sat['activation_date'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), pandasDF['activation_date'], np.nan)
temp_sat['activation_time'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), "Unknown", np.nan)
temp_sat['ticket_passengers'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), pandasDF['ticket_passengers'], np.nan)
temp_sat['Origin'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Destination'], np.nan)
temp_sat['Destination'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), pandasDF['Origin'], np.nan)
temp_sat['amount_paid_cad'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), pandasDF['amount_paid_cad'], np.nan)
temp_sat['Revenue'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), "0", np.nan)
temp_sat['Boardings'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), pandasDF['Boardings'], np.nan)

temp_sat['Service_Provider'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Service_Provider'], np.nan)
temp_sat['Ticket_System_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Ticket_System_Name'], np.nan)
temp_sat['Line_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Line_Name'], np.nan)
                                   
 

#df_list.concat(temp)
temp_sat=temp_sat[pd.notnull(temp_sat['ticket_code'])] 



# In[17]:


import pandas as pd 
import numpy as np 
from datetime import timedelta, date
temp_Sat_oneday = pd.DataFrame()
temp_Sat_oneday['ticket_code'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), "full_weekend_pass", np.nan)
temp_Sat_oneday['Universal_Class'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), "full_weekend_pass", np.nan)
temp_Sat_oneday['Year'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Year'], np.nan)
temp_Sat_oneday['Month'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Month'], np.nan)
temp_Sat_oneday['Day'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Day'], np.nan)
temp_Sat_oneday['Hour_24'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Hour_24'], np.nan)
temp_Sat_oneday['Minute'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Minute'], np.nan)
temp_Sat_oneday['FYear'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['FYear'], np.nan)
temp_Sat_oneday['Week'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Week'], np.nan)
temp_Sat_oneday['DayOfWeek'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['DayOfWeek'], np.nan)
temp_Sat_oneday['Holidays'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Holidays'], np.nan)
temp_Sat_oneday['Time_period'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Time_period'], np.nan)
temp_Sat_oneday['stop_lat'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['stop_lat'], np.nan)
temp_Sat_oneday['stop_lon'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['stop_lon'], np.nan)
temp_Sat_oneday['Mode_Type'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Mode_Type'], np.nan)
temp_Sat_oneday['activation_date'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['activation_date']+ pd.Timedelta(days=1), np.nan)
temp_Sat_oneday['activation_time'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), "Unknown", np.nan)
temp_Sat_oneday['ticket_passengers'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), pandasDF['ticket_passengers'], np.nan)
temp_Sat_oneday['Origin'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Origin'], np.nan)
temp_Sat_oneday['Destination'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), pandasDF['Destination'], np.nan)
temp_Sat_oneday['amount_paid_cad'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), pandasDF['amount_paid_cad'], np.nan)
temp_Sat_oneday['Revenue'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), "0", np.nan)
temp_Sat_oneday['Boardings'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), pandasDF['Boardings'], np.nan)

temp_Sat_oneday['Service_Provider'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Service_Provider'], np.nan)
temp_Sat_oneday['Ticket_System_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Ticket_System_Name'], np.nan)
temp_Sat_oneday['Line_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Line_Name'], np.nan)
                                   
 

#df_list.concat(temp)
temp_Sat_oneday=temp_Sat_oneday[pd.notnull(temp_Sat_oneday['ticket_code'])] 



# In[18]:


import pandas as pd 
import numpy as np 
from datetime import timedelta, date
temp_sat_oneday_R = pd.DataFrame()
temp_sat_oneday_R['ticket_code'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), "full_weekend_pass", np.nan)
temp_sat_oneday_R['Universal_Class'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), "full_weekend_pass", np.nan)
temp_sat_oneday_R['Year'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Year'], np.nan)
temp_sat_oneday_R['Month'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Month'], np.nan)
temp_sat_oneday_R['Day'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Day'], np.nan)
temp_sat_oneday_R['Hour_24'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Hour_24'], np.nan)
temp_sat_oneday_R['Minute'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Minute'], np.nan)
temp_sat_oneday_R['FYear'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['FYear'], np.nan)
temp_sat_oneday_R['Week'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Week'], np.nan)
temp_sat_oneday_R['DayOfWeek'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['DayOfWeek'], np.nan)
temp_sat_oneday_R['Holidays'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Holidays'], np.nan)
temp_sat_oneday_R['Time_period'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Time_period'], np.nan)
temp_sat_oneday_R['stop_lat'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['stop_lat'], np.nan)
temp_sat_oneday_R['stop_lon'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['stop_lon'], np.nan)
temp_sat_oneday_R['Mode_Type'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Mode_Type'], np.nan)
temp_sat_oneday_R['activation_date'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['activation_date']+ pd.Timedelta(days=1), np.nan)
#temp2['activation_date'] = np.where((pandasDF['Universal_Class'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1'), pandasDF['activation_date']+ Timedelta(days=1), np.nan)
temp_sat_oneday_R['activation_time'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), "Unknown", np.nan)
temp_sat_oneday_R['ticket_passengers'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), pandasDF['ticket_passengers'], np.nan)
temp_sat_oneday_R['Origin'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Destination'], np.nan)
temp_sat_oneday_R['dropoff_terminal'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), pandasDF['Origin'], np.nan)
temp_sat_oneday_R['amount_paid_cad'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), pandasDF['amount_paid_cad'], np.nan)
temp_sat_oneday_R['Revenue'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), "0", np.nan)
temp_sat_oneday_R['Boardings'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '7'), pandasDF['Boardings'], np.nan)

temp_sat_oneday_R['Service_Provider'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Service_Provider'], np.nan)
temp_sat_oneday_R['Ticket_System_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Ticket_System_Name'], np.nan)
temp_sat_oneday_R['Line_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '7'), pandasDF['Line_Name'], np.nan)
                                   
 

#df_list.concat(temp)
temp_sat_oneday_R=temp_sat_oneday_R[pd.notnull(temp_sat_oneday_R['ticket_code'])] 



# In[19]:


#Activation on Sunday and Monday is holiday 
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np 
temp_Sun_Mon = pd.DataFrame()
temp_Sun_Mon['ticket_code'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), "full_weekend_pass", np.nan)

temp_Sun_Mon['Universal_Class'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), "full_weekend_pass", np.nan)

temp_Sun_Mon['Universal_Class'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), "full_weekend_pass", np.nan)


temp_Sun_Mon['Year'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Year'], np.nan)

temp_Sun_Mon['Month'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Month'], np.nan)

temp_Sun_Mon['Day'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Day'], np.nan)

temp_Sun_Mon['Hour_24'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Hour_24'], np.nan)

temp_Sun_Mon['Minute'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Minute'], np.nan)

temp_Sun_Mon['FYear'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['FYear'], np.nan)

temp_Sun_Mon['Week'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Week'], np.nan)

temp_Sun_Mon['DayOfWeek'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['DayOfWeek'], np.nan)

temp_Sun_Mon['Holidays'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Holidays'], np.nan)

temp_Sun_Mon['Time_period'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Time_period'], np.nan)

temp_Sun_Mon['stop_lat'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['stop_lat'], np.nan)

temp_Sun_Mon['stop_lon'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['stop_lon'], np.nan)

temp_Sun_Mon['Mode_Type'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Mode_Type'], np.nan)

temp_Sun_Mon['activation_date'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['activation_date'], np.nan)

temp_Sun_Mon['activation_time'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), "Unknown", np.nan)

temp_Sun_Mon['ticket_passengers'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['ticket_passengers'], np.nan)

temp_Sun_Mon['Origin'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Destination'], np.nan)

temp_Sun_Mon['Destination'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Origin'], np.nan)

temp_Sun_Mon['amount_paid_cad'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['amount_paid_cad'], np.nan)

temp_Sun_Mon['Revenue'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), "0", np.nan)

temp_Sun_Mon['Boardings'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Boardings'], np.nan)

temp_Sun_Mon['Service_Provider'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Service_Provider'], np.nan)

temp_Sun_Mon['Ticket_System_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Ticket_System_Name'], np.nan)

temp_Sun_Mon['Line_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Line_Name'], np.nan)

                                   
 

#df_list.concat(temp)
temp_Sun_Mon=temp_Sun_Mon[pd.notnull(temp_Sun_Mon['ticket_code'])] 



# In[20]:


#Activation on Sunday and Monday is holiday
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np 
temp_Sun_Mon_oneday = pd.DataFrame()
temp_Sun_Mon_oneday['ticket_code'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), "full_weekend_pass", np.nan)
temp_Sun_Mon_oneday['Universal_Class'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), "full_weekend_pass", np.nan)


temp_Sun_Mon_oneday['Year'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Year'], np.nan)

temp_Sun_Mon_oneday['Month'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Month'], np.nan)

temp_Sun_Mon_oneday['Day'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Day'], np.nan)

temp_Sun_Mon_oneday['Hour_24'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Hour_24'], np.nan)

temp_Sun_Mon_oneday['Minute'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Minute'], np.nan)

temp_Sun_Mon_oneday['FYear'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['FYear'], np.nan)

temp_Sun_Mon_oneday['Week'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Week'], np.nan)

temp_Sun_Mon_oneday['DayOfWeek'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['DayOfWeek'], np.nan)

temp_Sun_Mon_oneday['Holidays'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Holidays'], np.nan)

temp_Sun_Mon_oneday['Time_period'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Time_period'], np.nan)

temp_Sun_Mon_oneday['stop_lat'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['stop_lat'], np.nan)

temp_Sun_Mon_oneday['stop_lon'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['stop_lon'], np.nan)

temp_Sun_Mon_oneday['Mode_Type'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Mode_Type'], np.nan)

temp_Sun_Mon_oneday['activation_date'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['activation_date']+pd.Timedelta(days=1), np.nan)

temp_Sun_Mon_oneday['activation_time'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), "Unknown", np.nan)

temp_Sun_Mon_oneday['ticket_passengers'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['ticket_passengers'], np.nan)

temp_Sun_Mon_oneday['Origin'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Origin'], np.nan)

temp_Sun_Mon_oneday['Destination'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Destination'], np.nan)

temp_Sun_Mon_oneday['amount_paid_cad'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['amount_paid_cad'], np.nan)

temp_Sun_Mon_oneday['Revenue'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), "0", np.nan)

temp_Sun_Mon_oneday['Boardings'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Boardings'], np.nan)

temp_Sun_Mon_oneday['Service_Provider'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Service_Provider'], np.nan)

temp_Sun_Mon_oneday['Ticket_System_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Ticket_System_Name'], np.nan)

temp_Sun_Mon_oneday['Line_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Line_Name'], np.nan)

                                   
 

#df_list.concat(temp)
temp_Sun_Mon_oneday=temp_Sun_Mon_oneday[pd.notnull(temp_Sun_Mon_oneday['ticket_code'])] 

In [ ]:
#Activation on Sunday and Monday is holiday
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np 
temp_Sun_Mon_oneday_R = pd.DataFrame()
temp_Sun_Mon_oneday_R['ticket_code'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), "full_weekend_pass", np.nan)

temp_Sun_Mon_oneday_R['Universal_Class'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), "full_weekend_pass", np.nan)


temp_Sun_Mon_oneday_R['Year'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Year'], np.nan)

temp_Sun_Mon_oneday_R['Month'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Month'], np.nan)

temp_Sun_Mon_oneday_R['Day'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Day'], np.nan)

temp_Sun_Mon_oneday_R['Hour_24'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Hour_24'], np.nan)

temp_Sun_Mon_oneday_R['Minute'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Minute'], np.nan)

temp_Sun_Mon_oneday_R['FYear'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['FYear'], np.nan)

temp_Sun_Mon_oneday_R['Week'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Week'], np.nan)

temp_Sun_Mon_oneday_R['DayOfWeek'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['DayOfWeek'], np.nan)

temp_Sun_Mon_oneday_R['Holidays'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Holidays'], np.nan)

temp_Sun_Mon_oneday_R['Time_period'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Time_period'], np.nan)

temp_Sun_Mon_oneday_R['stop_lat'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['stop_lat'], np.nan)

temp_Sun_Mon_oneday_R['stop_lon'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['stop_lon'], np.nan)

temp_Sun_Mon_oneday_R['Mode_Type'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Mode_Type'], np.nan)

temp_Sun_Mon_oneday_R['activation_date'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['activation_date']+pd.Timedelta(days=1), np.nan)

temp_Sun_Mon_oneday_R['activation_time'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), "Unknown", np.nan)

temp_Sun_Mon_oneday_R['ticket_passengers'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['ticket_passengers'], np.nan)

temp_Sun_Mon_oneday_R['Origin'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Destination'], np.nan)

temp_Sun_Mon_oneday_R['Destination'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Origin'], np.nan)

temp_Sun_Mon_oneday_R['amount_paid_cad'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['amount_paid_cad'], np.nan)

temp_Sun_Mon_oneday_R['Revenue'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), "0", np.nan)

temp_Sun_Mon_oneday_R['Boardings'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Boardings'], np.nan)

temp_Sun_Mon_oneday_R['Service_Provider'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Service_Provider'], np.nan)

temp_Sun_Mon_oneday_R['Ticket_System_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Ticket_System_Name'], np.nan)

temp_Sun_Mon_oneday_R['Line_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays']), pandasDF['Line_Name'], np.nan)

                                   
 

#df_list.concat(temp)
temp_Sun_Mon_oneday_R=temp_Sun_Mon_oneday_R[pd.notnull(temp_Sun_Mon_oneday_R['ticket_code'])]



# In[22]:


#Activation on Sunday and Monday is not in holiday
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np 
temp_Sun_Mon_NoHoli = pd.DataFrame()
temp_Sun_Mon_NoHoli['ticket_code'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), "full_weekend_pass", np.nan)

temp_Sun_Mon_NoHoli['Universal_Class'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), "full_weekend_pass", np.nan)

temp_Sun_Mon_NoHoli['Year'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['Year'], np.nan)

temp_Sun_Mon_NoHoli['Month'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['Month'], np.nan)

temp_Sun_Mon_NoHoli['Day'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['Day'], np.nan)

temp_Sun_Mon_NoHoli['Hour_24'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['Hour_24'], np.nan)

temp_Sun_Mon_NoHoli['Minute'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['Minute'], np.nan)

temp_Sun_Mon_NoHoli['FYear'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['FYear'], np.nan)

temp_Sun_Mon_NoHoli['Week'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['Week'], np.nan)

temp_Sun_Mon_NoHoli['DayOfWeek'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['DayOfWeek'], np.nan)

temp_Sun_Mon_NoHoli['Holidays'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['Holidays'], np.nan)

temp_Sun_Mon_NoHoli['Time_period'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['Time_period'], np.nan)

temp_Sun_Mon_NoHoli['stop_lat'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['stop_lat'], np.nan)

temp_Sun_Mon_NoHoli['stop_lon'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['stop_lon'], np.nan)

temp_Sun_Mon_NoHoli['Mode_Type'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['Mode_Type'], np.nan)

temp_Sun_Mon_NoHoli['activation_date'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['activation_date'], np.nan)

temp_Sun_Mon_NoHoli['activation_time'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), "Unknown", np.nan)

temp_Sun_Mon_NoHoli['ticket_passengers'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['ticket_passengers'], np.nan)

temp_Sun_Mon_NoHoli['Origin'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['Destination'], np.nan)

temp_Sun_Mon_NoHoli['Destination'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['Origin'], np.nan)

temp_Sun_Mon_NoHoli['amount_paid_cad'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['amount_paid_cad'], np.nan)

temp_Sun_Mon_NoHoli['Revenue'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), "0", np.nan)

temp_Sun_Mon_NoHoli['Boardings'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['Boardings'], np.nan)

temp_Sun_Mon_NoHoli['Service_Provider'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['Service_Provider'], np.nan)

temp_Sun_Mon_NoHoli['Ticket_System_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['Ticket_System_Name'], np.nan)

temp_Sun_Mon_NoHoli['Line_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['DayOfWeek'] == '1') & (~(pandasDF['activation_date']+pd.Timedelta(days=1)).isin(pandas_holiday['Holidays'])), pandasDF['Line_Name'], np.nan)

                                   
 

#df_list.concat(temp)
temp_Sun_Mon_NoHoli=temp_Sun_Mon_NoHoli[pd.notnull(temp_Sun_Mon_NoHoli['ticket_code'])]



# In[23]:


#Part-B/Logic for Monday Holidays
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np 
temp_Mon_Holi = pd.DataFrame()
temp_Mon_Holi['ticket_code'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), "full_weekend_pass", np.nan)
temp_Mon_Holi['Universal_Class'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), "full_weekend_pass", np.nan)
temp_Mon_Holi['Year'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['Year'], np.nan)
temp_Mon_Holi['Month'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['Month'], np.nan)
temp_Mon_Holi['Day'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['Day'], np.nan)
temp_Mon_Holi['Hour_24'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['Hour_24'], np.nan)
temp_Mon_Holi['Minute'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['Minute'], np.nan)
temp_Mon_Holi['FYear'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['FYear'], np.nan)
temp_Mon_Holi['Week'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['Week'], np.nan)
temp_Mon_Holi['DayOfWeek'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['DayOfWeek'], np.nan)
temp_Mon_Holi['Holidays'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['Holidays'], np.nan)
temp_Mon_Holi['Time_period'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['Time_period'], np.nan)
temp_Mon_Holi['stop_lat'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['stop_lat'], np.nan)
temp_Mon_Holi['stop_lon'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['stop_lon'], np.nan)
temp_Mon_Holi['Mode_Type'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['Mode_Type'], np.nan)
temp_Mon_Holi['activation_date'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['activation_date'], np.nan)
temp_Mon_Holi['activation_time'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), "Unknown", np.nan)
temp_Mon_Holi['ticket_passengers'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['ticket_passengers'], np.nan)
temp_Mon_Holi['Origin'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['Destination'], np.nan)
temp_Mon_Holi['Destination'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['Origin'], np.nan)
temp_Mon_Holi['amount_paid_cad'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['amount_paid_cad'], np.nan)
temp_Mon_Holi['Revenue'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), "0", np.nan)
temp_Mon_Holi['Boardings'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['Boardings'], np.nan)

temp_Mon_Holi['Service_Provider'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass')& (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['Service_Provider'], np.nan)
temp_Mon_Holi['Ticket_System_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['Ticket_System_Name'], np.nan)
temp_Mon_Holi['Line_Name'] = np.where((pandasDF['ticket_code'] == 'full_weekend_pass') & (pandasDF['Holidays'] != "null") & (pandasDF['DayOfWeek'] == '2'), pandasDF['Line_Name'], np.nan)

                                   
 

#df_list.concat(temp)
temp_Mon_Holi=temp_Mon_Holi[pd.notnull(temp_Mon_Holi['ticket_code'])]



# In[24]:


#Part-B/Logic for Weekdaygrouppass
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np 
weekday_group_pass_2 = pd.DataFrame()
weekday_group_pass_2['ticket_code'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), "weekday_group_pass_2_pax", np.nan)
weekday_group_pass_2['Universal_Class'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), "weekday_group_pass_2_pax", np.nan)
weekday_group_pass_2['Year'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['Year'], np.nan)
weekday_group_pass_2['Month'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['Month'], np.nan)
weekday_group_pass_2['Day'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['Day'], np.nan)
weekday_group_pass_2['Hour_24'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['Hour_24'], np.nan)
weekday_group_pass_2['Minute'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['Minute'], np.nan)
weekday_group_pass_2['FYear'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['FYear'], np.nan)
weekday_group_pass_2['Week'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['Week'], np.nan)
weekday_group_pass_2['DayOfWeek'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['DayOfWeek'], np.nan)
weekday_group_pass_2['Holidays'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['Holidays'], np.nan)
weekday_group_pass_2['Time_period'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['Time_period'], np.nan)
weekday_group_pass_2['stop_lat'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['stop_lat'], np.nan)
weekday_group_pass_2['stop_lon'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['stop_lon'], np.nan)
weekday_group_pass_2['Mode_Type'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['Mode_Type'], np.nan)
weekday_group_pass_2['activation_date'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['activation_date'], np.nan)
weekday_group_pass_2['activation_time'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), "Unknown", np.nan)
weekday_group_pass_2['ticket_passengers'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['ticket_passengers'], np.nan)
weekday_group_pass_2['Origin'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['Destination'], np.nan)
weekday_group_pass_2['Destination'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['Origin'], np.nan)
weekday_group_pass_2['amount_paid_cad'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['amount_paid_cad'], np.nan)
weekday_group_pass_2['Revenue'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), "0", np.nan)
weekday_group_pass_2['Boardings'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['Boardings'], np.nan)

weekday_group_pass_2['Service_Provider'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['Service_Provider'], np.nan)
weekday_group_pass_2['Ticket_System_Name'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['Ticket_System_Name'], np.nan)
weekday_group_pass_2['Line_Name'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_2_pax'), pandasDF['Line_Name'], np.nan)

                                   
 

#df_list.concat(temp)
weekday_group_pass_2=weekday_group_pass_2[pd.notnull(weekday_group_pass_2['ticket_code'])]



# In[25]:


#Part-B/Logic for Weekdaygrouppass
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np 
weekday_group_pass_3 = pd.DataFrame()
weekday_group_pass_3['ticket_code'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), "weekday_group_pass_3_pax", np.nan)
weekday_group_pass_3['Universal_Class'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), "weekday_group_pass_3_pax", np.nan)
weekday_group_pass_3['Year'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['Year'], np.nan)
weekday_group_pass_3['Month'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['Month'], np.nan)
weekday_group_pass_3['Day'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['Day'], np.nan)
weekday_group_pass_3['Hour_24'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['Hour_24'], np.nan)
weekday_group_pass_3['Minute'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['Minute'], np.nan)
weekday_group_pass_3['FYear'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['FYear'], np.nan)
weekday_group_pass_3['Week'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['Week'], np.nan)
weekday_group_pass_3['DayOfWeek'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['DayOfWeek'], np.nan)
weekday_group_pass_3['Holidays'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['Holidays'], np.nan)
weekday_group_pass_3['Time_period'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['Time_period'], np.nan)
weekday_group_pass_3['stop_lat'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['stop_lat'], np.nan)
weekday_group_pass_3['stop_lon'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['stop_lon'], np.nan)
weekday_group_pass_3['Mode_Type'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['Mode_Type'], np.nan)
weekday_group_pass_3['activation_date'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['activation_date'], np.nan)
weekday_group_pass_3['activation_time'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), "Unknown", np.nan)
weekday_group_pass_3['ticket_passengers'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['ticket_passengers'], np.nan)
weekday_group_pass_3['Origin'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['Destination'], np.nan)
weekday_group_pass_3['Destination'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['Origin'], np.nan)
weekday_group_pass_3['amount_paid_cad'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['amount_paid_cad'], np.nan)
weekday_group_pass_3['Revenue'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), "0", np.nan)
weekday_group_pass_3['Boardings'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['Boardings'], np.nan)

weekday_group_pass_3['Service_Provider'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['Service_Provider'], np.nan)
weekday_group_pass_3['Ticket_System_Name'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['Ticket_System_Name'], np.nan)
weekday_group_pass_3['Line_Name'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_3_pax'), pandasDF['Line_Name'], np.nan)

                                   
 

#df_list.concat(temp)
weekday_group_pass_3=weekday_group_pass_3[pd.notnull(weekday_group_pass_3['ticket_code'])]

In [ ]:
#Part-B/Logic for Weekdaygrouppass
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np 
weekday_group_pass_4 = pd.DataFrame()
weekday_group_pass_4['ticket_code'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), "weekday_group_pass_4_pax", np.nan)
weekday_group_pass_4['Universal_Class'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), "weekday_group_pass_4_pax", np.nan)
weekday_group_pass_4['Year'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['Year'], np.nan)
weekday_group_pass_4['Month'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['Month'], np.nan)
weekday_group_pass_4['Day'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['Day'], np.nan)
weekday_group_pass_4['Hour_24'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['Hour_24'], np.nan)
weekday_group_pass_4['Minute'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['Minute'], np.nan)
weekday_group_pass_4['FYear'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['FYear'], np.nan)
weekday_group_pass_4['Week'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['Week'], np.nan)
weekday_group_pass_4['DayOfWeek'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['DayOfWeek'], np.nan)
weekday_group_pass_4['Holidays'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['Holidays'], np.nan)
weekday_group_pass_4['Time_period'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['Time_period'], np.nan)
weekday_group_pass_4['stop_lat'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['stop_lat'], np.nan)
weekday_group_pass_4['stop_lon'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['stop_lon'], np.nan)
weekday_group_pass_4['Mode_Type'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['Mode_Type'], np.nan)
weekday_group_pass_4['activation_date'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['activation_date'], np.nan)
weekday_group_pass_4['activation_time'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), "Unknown", np.nan)
weekday_group_pass_4['ticket_passengers'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['ticket_passengers'], np.nan)
weekday_group_pass_4['Origin'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['Destination'], np.nan)
weekday_group_pass_4['Destination'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['Origin'], np.nan)
weekday_group_pass_4['amount_paid_cad'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['amount_paid_cad'], np.nan)
weekday_group_pass_4['Revenue'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), "0", np.nan)
weekday_group_pass_4['Boardings'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['Boardings'], np.nan)

weekday_group_pass_4['Service_Provider'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['Service_Provider'], np.nan)
weekday_group_pass_4['Ticket_System_Name'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['Ticket_System_Name'], np.nan)
weekday_group_pass_4['Line_Name'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_4_pax'), pandasDF['Line_Name'], np.nan)

                            
 

#df_list.concat(temp)
weekday_group_pass_4=weekday_group_pass_4[pd.notnull(weekday_group_pass_4['ticket_code'])]

In [ ]:
#Part-B/Logic for Weekdaygrouppass
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np 
weekday_group_pass_5 = pd.DataFrame()
weekday_group_pass_5['ticket_code'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), "weekday_group_pass_5_pax", np.nan)
weekday_group_pass_5['Universal_Class'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), "weekday_group_pass_5_pax", np.nan)
weekday_group_pass_5['Year'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['Year'], np.nan)
weekday_group_pass_5['Month'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['Month'], np.nan)
weekday_group_pass_5['Day'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['Day'], np.nan)
weekday_group_pass_5['Hour_24'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['Hour_24'], np.nan)
weekday_group_pass_5['Minute'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['Minute'], np.nan)
weekday_group_pass_5['FYear'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['FYear'], np.nan)
weekday_group_pass_5['Week'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['Week'], np.nan)
weekday_group_pass_5['DayOfWeek'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['DayOfWeek'], np.nan)
weekday_group_pass_5['Holidays'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['Holidays'], np.nan)
weekday_group_pass_5['Time_period'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['Time_period'], np.nan)
weekday_group_pass_5['stop_lat'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['stop_lat'], np.nan)
weekday_group_pass_5['stop_lon'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['stop_lon'], np.nan)
weekday_group_pass_5['Mode_Type'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['Mode_Type'], np.nan)
weekday_group_pass_5['activation_date'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['activation_date'], np.nan)
weekday_group_pass_5['activation_time'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), "Unknown", np.nan)
weekday_group_pass_5['ticket_passengers'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['ticket_passengers'], np.nan)
weekday_group_pass_5['Origin'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['Destination'], np.nan)
weekday_group_pass_5['Destination'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['Origin'], np.nan)
weekday_group_pass_5['amount_paid_cad'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['amount_paid_cad'], np.nan)
weekday_group_pass_5['Revenue'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), "0", np.nan)
weekday_group_pass_5['Boardings'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['Boardings'], np.nan)

weekday_group_pass_5['Service_Provider'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['Service_Provider'], np.nan)
weekday_group_pass_5['Ticket_System_Name'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['Ticket_System_Name'], np.nan)
weekday_group_pass_5['Line_Name'] = np.where((pandasDF['ticket_code'] == 'weekday_group_pass_5_pax'), pandasDF['Line_Name'], np.nan)

                              
 

#df_list.concat(temp)
weekday_group_pass_5=weekday_group_pass_5[pd.notnull(weekday_group_pass_5['ticket_code'])]

In [ ]:
Mozio_presto_df=pd.concat([pandasDF,temp_Fri,temp_Fri_oneday,temp_Fri_oneday_R,temp_sat,temp_Sat_oneday,temp_sat_oneday_R,temp_Sun_Mon,temp_Sun_Mon_oneday,temp_Sun_Mon_oneday_R,temp_Sun_Mon_NoHoli,temp_Mon_Holi,weekday_group_pass_2,weekday_group_pass_3,weekday_group_pass_4,weekday_group_pass_5])

In [ ]:
Mozio_presto_df1= Mozio_presto_df[Mozio_presto_df.Origin != "null"]
boarding_columns = ['Year','Month','Day','Hour_24','Minute','FYear','Week','Time_period','Service_Provider','Universal_Class','Ticket_System_Name','Origin','stop_lat','stop_lon','Line_Name','Mode_Type','Boardings','Revenue']
pd = Mozio_presto_df1[boarding_columns]

In [ ]:
final = spark.createDataFrame(pd)

In [ ]:
#import os

#from datetime import datetime
#date = datetime.now().strftime("%Y-%m-%d") 

#target_folder_path = 'abfss://silver@mxdatalakedevcc001.dfs.core.windows.net/digital/mozio/Mozio_Presto_Output/BoardingNotebook_Output'

#BoardingFileName = 'Mozio_Presto_Output_' + date + '.parquet'
#Full_target_path = os.path.join(target_folder_path, BoardingFileName ) 

In [ ]:
final.coalesce(1).write.format(lpaOutputFormat).mode(lpaOutputMode).save(targetFolderPath)